In [1]:
import os
import sys
import pandas as pd
from lib.display import Display
from lib.database import Database
from lib.schema import Schema

db = Database(Schema.ACCEPT)   
Display.header('Accept study on ALP', 'h1')

Display.header('Participants', 'h3')
participants_df = db.get_participants()
Display.frame(participants_df, 5)
Display.header('Responses', 'h3')
responses_df = db.get_responses()
Display.frame(responses_df, 5)

print('Number of users:', participants_df.shape[0])
print('Number of responses:', responses_df.shape[0])
print(f'Last donation on: {responses_df["AUTHORED"].max().strftime("%d.%m.%Y")}')

,ALP_ID,EXTERNAL_ID,STATUS,START_DATE,END_DATE
1,10999,ULM-000201,enrolled,2022-02-13,NaT
2,12961,ULM-000203,enrolled,2022-04-04,NaT
3,12707,ULM-000217,enrolled,2022-03-28,NaT
4,11004,ULM-000228,enrolled,2022-02-09,NaT
5,9553,ULM-000214,enrolled,2022-01-13,NaT


,ALP_ID,VERSION,AUTHORED,QUESTIONNAIRE,LINK_ID,VALUE,VALUECODING_CODE,LANGUAGE,TEXT,QUESTIONNAIRE_ID
1,9553,1.0.0,2022-01-13,Q1,what_birth_year,1977,NaN,de,In welchem Jahr sind Sie geboren?,Cqhhfx6laM+L/5Bi+cgg/dggN1gQurQdWoSfZBwBO0ITd093hGOfkh6mWN9kboQN6FjnLtW1FmzzXE1anZWjAVTGjS40Om0x4AzSb94Vzpas8o8OFmnbQOic/Hv/Kz6BWvUu9dTBJHYpCYvlH6g0kciUC4lYTayvLsYs07BHCl4iPvUQw63TzdnfGbrxO3Ni4mouVGTmj4inJdid9YXM2179j8dhhmnB9pX2ayA0luu8S6yITRvIw2jzkOcL9kOMGPfi15Vo5dRicyMZBsp/aqRyHUcEF1zHreiUBRowmqqrMOyF4654aUSQEYUx0Y/eBziiwmb1L5duJb9GOnXxDg==
2,9553,1.0.0,2022-01-13,Q1,what_gender,NaN,female,de,Welchem Geschlecht ordnen Sie sich zu?,Cqhhfx6laM+L/5Bi+cgg/dggN1gQurQdWoSfZBwBO0ITd093hGOfkh6mWN9kboQN6FjnLtW1FmzzXE1anZWjAVTGjS40Om0x4AzSb94Vzpas8o8OFmnbQOic/Hv/Kz6BWvUu9dTBJHYpCYvlH6g0kciUC4lYTayvLsYs07BHCl4iPvUQw63TzdnfGbrxO3Ni4mouVGTmj4inJdid9YXM2179j8dhhmnB9pX2ayA0luu8S6yITRvIw2jzkOcL9kOMGPfi15Vo5dRicyMZBsp/aqRyHUcEF1zHreiUBRowmqqrMOyF4654aUSQEYUx0Y/eBziiwmb1L5duJb9GOnXxDg==
3,9553,1.0.0,2022-01-13,Q1,what_height,172,NaN,de,Wie groÃ sind Sie (cm)?,Cqhhfx6laM+L/5Bi+cgg/dggN1gQurQdWoSfZBwBO0ITd093hGOfkh6mWN9kboQN6FjnLtW1FmzzXE1anZWjAVTGjS40Om0x4AzSb94Vzpas8o8OFmnbQOic/Hv/Kz6BWvUu9dTBJHYpCYvlH6g0kciUC4lYTayvLsYs07BHCl4iPvUQw63TzdnfGbrxO3Ni4mouVGTmj4inJdid9YXM2179j8dhhmnB9pX2ayA0luu8S6yITRvIw2jzkOcL9kOMGPfi15Vo5dRicyMZBsp/aqRyHUcEF1zHreiUBRowmqqrMOyF4654aUSQEYUx0Y/eBziiwmb1L5duJb9GOnXxDg==
4,9553,1.0.0,2022-01-13,Q1,what_weight,109,NaN,de,Was ist Ihr KÃ¶rpergewicht (kg)?,Cqhhfx6laM+L/5Bi+cgg/dggN1gQurQdWoSfZBwBO0ITd093hGOfkh6mWN9kboQN6FjnLtW1FmzzXE1anZWjAVTGjS40Om0x4AzSb94Vzpas8o8OFmnbQOic/Hv/Kz6BWvUu9dTBJHYpCYvlH6g0kciUC4lYTayvLsYs07BHCl4iPvUQw63TzdnfGbrxO3Ni4mouVGTmj4inJdid9YXM2179j8dhhmnB9pX2ayA0luu8S6yITRvIw2jzkOcL9kOMGPfi15Vo5dRicyMZBsp/aqRyHUcEF1zHreiUBRowmqqrMOyF4654aUSQEYUx0Y/eBziiwmb1L5duJb9GOnXxDg==
5,9553,1.0.0,2022-01-13,Q1,when_is_operation,2022-01-28,NaN,de,Wann ist Ihre Darmoperation vorgesehen?,Cqhhfx6laM+L/5Bi+cgg/dggN1gQurQdWoSfZBwBO0ITd093hGOfkh6mWN9kboQN6FjnLtW1FmzzXE1anZWjAVTGjS40Om0x4AzSb94Vzpas8o8OFmnbQOic/Hv/Kz6BWvUu9dTBJHYpCYvlH6g0kciUC4lYTayvLsYs07BHCl4iPvUQw63TzdnfGbrxO3Ni4mouVGTmj4inJdid9YXM2179j8dhhmnB9pX2ayA0luu8S6yITRvIw2jzkOcL9kOMGPfi15Vo5dRicyMZBsp/aqRyHUcEF1zHreiUBRowmqqrMOyF4654aUSQEYUx0Y/eBziiwmb1L5duJb9GOnXxDg==


Number of users: 6
Number of responses: 562
Last donation on: 15.04.2022


In [2]:
from lib.studies.accept_study import AcceptStudy 
from lib.participant_builder import ParticipantsBuilder
from datetime import datetime, timedelta
import pandas as pd
df = ParticipantsBuilder(AcceptStudy(), participants_df, responses_df) \
    .add_last_donation() \
    .add_all_submissions() \
    .add_answer('Q1','when_is_operation', 'OPERATION_DATE') \
    .add_answer('Q1','what_birth_year', 'BIRTHDATE') \
    .get()

Display.frame(df)

df['OPERATION_DATE'] = pd.to_datetime(df['OPERATION_DATE'])
df['AGE'] = ((datetime.today() - pd.to_datetime(df['BIRTHDATE'])).dt.days / 365).astype(int)
df['DAYS_PASSED'] = (datetime.today() - pd.to_datetime(df['OPERATION_DATE'])).dt.days
df = df.sort_values(by=['START_DATE'], ascending=False)

Display.header('Show all users', 'h2')
Display.frame(df)

df = responses_df[responses_df['LINK_ID'] == 'when_is_operation']
df = df.groupby(['ALP_ID']).size().reset_index(name='counts')
print(f'Users reported operation date more than once: {df[df["counts"] > 1].shape[0]}')


,ALP_ID,EXTERNAL_ID,STATUS,START_DATE,END_DATE,ENROLED_ON,LAST_DONATION,all,Q1,Q2,Q3,Q4a,Q4b,Q6,Q7,Q8,Q9,OPERATION_DATE,BIRTHDATE
0,10999,ULM-000201,enrolled,2022-02-13,NaT,2022-02-13,2022-03-25,8,1,1,2,1,1,1,0,1,0,2022-02-16,1961
1,12961,ULM-000203,enrolled,2022-04-04,NaT,2022-04-04,2022-04-09,9,1,1,3,1,1,1,0,1,0,2022-04-05,1956
2,12707,ULM-000217,enrolled,2022-03-28,NaT,2022-03-28,2022-04-15,5,1,1,1,1,1,0,0,0,0,2022-04-20,1960
3,11004,ULM-000228,enrolled,2022-02-09,NaT,2022-02-09,2022-02-09,5,1,1,1,1,1,0,0,0,0,2022-02-11,1948
4,9553,ULM-000214,enrolled,2022-01-13,NaT,2022-01-13,2022-01-23,5,1,1,1,1,1,0,0,0,0,2022-01-28,1977
5,10208,ULM-000215,enrolled,2022-01-28,NaT,2022-01-28,2022-02-12,5,1,1,1,1,1,0,0,0,0,2022-02-17,1958


,ALP_ID,EXTERNAL_ID,STATUS,START_DATE,END_DATE,ENROLED_ON,LAST_DONATION,all,Q1,Q2,Q3,Q4a,Q4b,Q6,Q7,Q8,Q9,OPERATION_DATE,BIRTHDATE,AGE,DAYS_PASSED
1,12961,ULM-000203,enrolled,2022-04-04,NaT,2022-04-04,2022-04-09,9,1,1,3,1,1,1,0,1,0,2022-04-05,1956,66,198
2,12707,ULM-000217,enrolled,2022-03-28,NaT,2022-03-28,2022-04-15,5,1,1,1,1,1,0,0,0,0,2022-04-20,1960,62,183
0,10999,ULM-000201,enrolled,2022-02-13,NaT,2022-02-13,2022-03-25,8,1,1,2,1,1,1,0,1,0,2022-02-16,1961,61,246
3,11004,ULM-000228,enrolled,2022-02-09,NaT,2022-02-09,2022-02-09,5,1,1,1,1,1,0,0,0,0,2022-02-11,1948,74,251
5,10208,ULM-000215,enrolled,2022-01-28,NaT,2022-01-28,2022-02-12,5,1,1,1,1,1,0,0,0,0,2022-02-17,1958,64,245
4,9553,ULM-000214,enrolled,2022-01-13,NaT,2022-01-13,2022-01-23,5,1,1,1,1,1,0,0,0,0,2022-01-28,1977,45,265


Users reported operation date more than once: 0


In [3]:
import pandas as pd

df = pd.merge(responses_df, participants_df[['ALP_ID', 'EXTERNAL_ID']], how='left', on='ALP_ID')
df = df.groupby(['ALP_ID', 'EXTERNAL_ID','AUTHORED','QUESTIONNAIRE', 'QUESTIONNAIRE_ID']).count()
Display.frame(df)